INSTRUÇÕES:
- Substitua TODOS os {PLACEHOLDERS} pelos seus valores reais
- Crie External Location ANTES no Unity Catalog

1. CRIAR SCHEMA

In [0]:
%sql
-- Run once as admin to setup Unity Catalog resources
-- Replace placeholders: {catalog}, {schema}, {external_location}

CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}
COMMENT 'Schema for bikestore logistics project';

2. CRIAR EXTERNAL VOLUME (Volume mapeia storage)

In [0]:
%sql
CREATE EXTERNAL VOLUME IF NOT EXISTS {catalog}.{schema}.bikestore_resource
COMMENT 'Volume for bikestore resources - UPDATE LOCATION'
LOCATION '{external_location}/bikestore/resource';  -- e.g., abfss://container@storage.dfs.core.windows.net/path


3. VERIFICAR VOLUME (descomente após criar)

In [0]:
display(dbutils.fs.ls('/Volumes/{catalog_name}/{schema_name}/bikestore_resource/origem/'));

4. VERIFICAR EXTERNAL LOCATION DIRETO (descomente após configurar)

In [0]:
display(dbutils.fs.ls('{external_location}/bikestore/'));


5. DEFINIR VARIÁVEIS DE CAMADA (substitua {external_location})

In [0]:
bronze_path   = '{external_location}/bikestore/bronze/'
silver_path   = '{external_location}/bikestore/silver/'
gold_path     = '{external_location}/bikestore/gold/'
resource_path = '{external_location}/bikestore/resource/origem/'
resource_path_volume = f"/Volumes/{CATALOG_NAME}/{SCHEMA_NAME}/bikestore_resource/origem/"

print(f"Bronze:   {bronze_path}")
print(f"Silver:   {silver_path}")
print(f"Gold:     {gold_path}")
print(f"Resource: {resource_path}")

6. TESTAR ESTRUTURA (descomente após definir variáveis)

In [0]:
display(dbutils.fs.ls(bronze_path))

7. TESTE DE LEITURA (descomente se tiver brands.csv na origem)

In [0]:
df_sample = spark.read.csv(
    f'{resource_path}brands.csv',
    header=True,
    inferSchema=True,
    sep=','
)
display(df_sample.limit(5))

8. SALVAR PRIMEIRO ARQUIVO BRONZE (teste - descomente)

In [0]:
df_sample.write \
    .mode('overwrite') \
    .format('delta') \
    .option("mergeSchema", "true") \
    .save(f"{bronze_path}brands/")

9. CRIAR TABELA BRONZE (opcional - descomente)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.bronze_brands
USING DELTA
LOCATION '{external_location}/bikestore/bronze/brands/'
TBLPROPERTIES ("delta.autoOptimize.optimizeWrite" = "true");